## Model Training

#### Import Data and Required Packages

- Importing Pandas , Numpy , Matplotlib , Seaborn and Warnings Libraries

In [1]:
# Basic Import 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Modeling 
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

#### Import CSV Data as Pandas Dataframe

In [2]:
df = pd.read_csv('data/stud.csv')

#### Show Top 5 Records

In [4]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


#### Preparing X and Y Variables

In [5]:
X = df.drop(columns=['math_score'],axis=1)
Y = df[['math_score']]

In [6]:
X.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [7]:
Y.head()

,math_score
0,72
1,69
2,90
3,47
4,76


In [8]:
print("Categories in 'gender' variable :   ",end=" ")
print(df['gender'].unique())

print("Categories in 'race/ethnicity' variable :  ",end=" ")
print(df['race_ethnicity'].unique())

print("Categories in 'parental level of eduction' variable :   ",end=" ")
print(df['parental_level_of_education'].unique())

print("Categories in 'lunch' variable :   ",end=" ")
print(df['lunch'].unique())

print("Categories in 'test preparation course' variable :   ",end=" ")
print(df['test_preparation_course'].unique())

Categories in 'gender' variable :    ['female' 'male']
Categories in 'race/ethnicity' variable :   ['group B' 'group C' 'group A' 'group D' 'group E']
Categories in 'parental level of eduction' variable :    ["bachelor's degree" 'some college' "master's degree" "associate's degree"
 'high school' 'some high school']
Categories in 'lunch' variable :    ['standard' 'free/reduced']
Categories in 'test preparation course' variable :    ['none' 'completed']


In [13]:
# Create Column Transformer with 3 types of Tranfomers
num_features = X.select_dtypes(exclude='object').columns
cat_features = X.select_dtypes(include='object').columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ('OneHotEncoder',oh_transformer,cat_features),
        ('StandardScaler',numeric_transformer,num_features)
    ]
)

In [16]:
X = preprocessor.fit_transform(X)

In [18]:
X.shape

(1000, 19)

In [19]:
# Seperate Data into Training and Testing Datsets
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2,random_state=42)
X_train.shape , Y_test.shape

((800, 19), (200, 1))

#### Create an Evaluation Function to give all metrics after model Training

In [21]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true,predicted))
    r2_square = r2_score(true,predicted)
    return mae,rmse,r2_square

In [23]:
models = {
    "LinearRegression" : LinearRegression(),
    "Lasso" : Lasso(),
    "Ridge" : Ridge(),
    "K-Neighbour Regressor" : KNeighborsRegressor(),
    "Decision Tree" : DecisionTreeRegressor(),
    "Random Forest Regressor" : RandomForestRegressor(),
    "XGBRegressor" : XGBRegressor(),
    "CatBoosting Regressor" : CatBoostRegressor(),
    "AdaBoost Regressor" : AdaBoostRegressor()
}

model_list = []
r2_list = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train,Y_train)

    # Make Prediction :
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Evaluation Train and Test Dataset
    model_train_mae , model_train_rmse , model_train_r2 = evaluate_model(Y_train,y_train_pred)
    model_test_mae , model_test_rmse , model_test_r2 = evaluate_model(Y_test , y_test_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Performance for Training Set :- ')
    print(" Root Mean Square Error : {:.4f}".format(model_train_rmse))
    print(" Mean Absolute Error : {:.4f}".format(model_train_mae))
    print(" R2 Score : {:.4f}".format(model_train_r2))

    print("-------------------------------------------")

    print('Model Performance for Testing Set :- ')
    print(" Root Mean Square Error : {:.4f}".format(model_test_rmse))
    print(" Mean Absolute Error : {:.4f}".format(model_test_mae))
    print(" R2 Score : {:.4f}".format(model_test_r2))

    r2_list.append(model_test_r2)

    print('*'*35)
    print('\n')

LinearRegression
Model Performance for Training Set :- 
 Root Mean Square Error : 5.3243
 Mean Absolute Error : 4.2671
 R2 Score : 0.8743
-------------------------------------------
Model Performance for Testing Set :- 
 Root Mean Square Error : 5.3960
 Mean Absolute Error : 4.2158
 R2 Score : 0.8803
***********************************


Lasso
Model Performance for Training Set :- 
 Root Mean Square Error : 6.5938
 Mean Absolute Error : 5.2063
 R2 Score : 0.8071
-------------------------------------------
Model Performance for Testing Set :- 
 Root Mean Square Error : 6.5197
 Mean Absolute Error : 5.1579
 R2 Score : 0.8253
***********************************


Ridge
Model Performance for Training Set :- 
 Root Mean Square Error : 5.3233
 Mean Absolute Error : 4.2650
 R2 Score : 0.8743
-------------------------------------------
Model Performance for Testing Set :- 
 Root Mean Square Error : 5.3904
 Mean Absolute Error : 4.2111
 R2 Score : 0.8806
***********************************


K-

c:\Users\mohit\OneDrive\Desktop\ML,AI\venv\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Random Forest Regressor
Model Performance for Training Set :- 
 Root Mean Square Error : 2.3129
 Mean Absolute Error : 1.8385
 R2 Score : 0.9763
-------------------------------------------
Model Performance for Testing Set :- 
 Root Mean Square Error : 6.0477
 Mean Absolute Error : 4.6578
 R2 Score : 0.8497
***********************************


XGBRegressor
Model Performance for Training Set :- 
 Root Mean Square Error : 1.0073
 Mean Absolute Error : 0.6875
 R2 Score : 0.9955
-------------------------------------------
Model Performance for Testing Set :- 
 Root Mean Square Error : 6.4733
 Mean Absolute Error : 5.0577
 R2 Score : 0.8278
***********************************


Learning rate set to 0.039525
0:	learn: 14.5987177	total: 1.71ms	remaining: 1.71s
1:	learn: 14.2251886	total: 3.17ms	remaining: 1.58s
2:	learn: 13.8866124	total: 4.46ms	remaining: 1.48s
3:	learn: 13.5235688	total: 5.74ms	remaining: 1.43s
4:	learn: 13.1887021	total: 12.3ms	remaining: 2.46s
5:	learn: 12.9124226	total:

c:\Users\mohit\OneDrive\Desktop\ML,AI\venv\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


AdaBoost Regressor
Model Performance for Training Set :- 
 Root Mean Square Error : 5.8307
 Mean Absolute Error : 4.7818
 R2 Score : 0.8492
-------------------------------------------
Model Performance for Testing Set :- 
 Root Mean Square Error : 6.0551
 Mean Absolute Error : 4.7035
 R2 Score : 0.8493
***********************************


